# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *A non-local algorithm for image denoising* autorstwa Antoni Buades, Bartomeu Coll, i Jean Michel Morel na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation}

gdzie:
- \begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in  V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation},
- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując. Jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/'

fileNames = ["MR_data.mat"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)


In [ ]:
def gaussian_filter(window_size, wariancja):
    kernel = np.fromfunction(
        lambda x, y: (1/(2*np.pi*wariancja**2)) * 
                     np.exp(-((x-(window_size-1)/2)**2 + (y-(window_size-1)/2)**2) / (2*wariancja**2)),
        (window_size, window_size)
    )
    return kernel / np.sum(kernel)

In [ ]:
def classic_conv(img, window, sigma):
    kernel = gaussian_filter(window, sigma)
    image = img.copy()
    height, width = image.shape
    img_with_padding = np.zeros((height + window - 1, width + window - 1))
    img_with_padding[(window - 1) // 2 : -(window - 1) // 2, (window - 1) // 2 : -(window - 1) // 2] = image

    output_img = np.zeros_like(image)

    for i in range(0, height):
        for j in range(0, width):
            output_img[i, j] = (kernel * img_with_padding[i : i + window, j : j + window]).sum()
    return output_img

In [ ]:

def new_pixel_value(img, context, window, sigma, x_, y_, gauss):
    new_window = 0
    new_value = 0
    for i in range(window):
        for j in range(window):
            light_diff = np.abs(int(context[i, j]) - int(img[x_, y_])).astype(np.int8)
            gamma = (
                np.exp((-1) * (((light_diff) ** 2) / (2 * (sigma**2))))
                * gauss[i, j]
                )
            new_window += gamma
            new_value += gamma * context[i, j]
    new_value = new_value / new_window if new_window != 0 else 0
    return new_value

def bilateral_conv(image, okno: int, sigma: float, sigma_r: float):
    height, width = image.shape
    result = np.zeros((height, width))
    size = okno // 2
    gauss = gaussian_filter(okno, sigma)
    for i in range(size, height - size):
        for j in range(size, width - size):
            context = image[i - size : i + size + 1, j - size : j + size + 1]
            result[i, j] = new_pixel_value(image, context, okno, sigma_r, i, j, gauss)
    return result

In [ ]:
def non_local_means(img, zakres_obszaru:int, okno: int, alpha: float, sigma: float):
    img = img.astype(np.float64)
    zakres_obszaru += okno // 2 * 2  
    height, width = img.shape
    call_gaus = lambda x: gaussian_filter(okno, sigma) * x
    image = np.zeros(img.shape)
    for x in range(zakres_obszaru // 2, height - zakres_obszaru // 2):
        for y in range(zakres_obszaru // 2, width - zakres_obszaru // 2):

            obszar_V = img[x - zakres_obszaru // 2 : x + zakres_obszaru // 2 + 1, y - zakres_obszaru // 2 : y + zakres_obszaru // 2 + 1]
            nowe_okno = np.array([])
            Nx = img[x - okno // 2 : x + okno // 2 + 1, y - okno // 2 : y + okno // 2 + 1]
            obszar_V_height, obszar_V_width = obszar_V.shape
            for s in range(okno // 2, obszar_V_height - okno // 2):
                for z in range(okno // 2, obszar_V_width - okno // 2):
                    
                    Np = obszar_V[s - okno // 2 : s + okno // 2 + 1, z - okno // 2 : z + okno // 2 + 1]
                    Wexp = np.exp(
                        -np.sum((call_gaus(Np) - call_gaus(Nx)) ** 2) / (alpha * sigma**2)
                    )
                    nowe_okno = np.append(nowe_okno, Wexp)
            W = nowe_okno / np.sum(nowe_okno)
            I = obszar_V[okno // 2 : -(okno // 2), okno // 2 : -(okno // 2)]
            image[x, y] = np.sum(W.reshape(I.shape) * I)
    max_ = max(image.flatten())

    return (image / max_ * 255).astype(np.uint8)

In [ ]:
#porownanie z pozostałymi funkcjami
MR_data = loadmat('MR_data.mat')
img = MR_data['I_noisy1'].copy()
_, ax = plt.subplots(1, 4, figsize=(15, 10))
ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original')
start_time = cv2.getTickCount()
ax[1].imshow(classic_conv(img, 7, 1), cmap='gray')
ax[1].set_title('Classic')
print('Classic convolution time: ', (cv2.getTickCount() - start_time) / cv2.getTickFrequency())
start_time = cv2.getTickCount()
ax[2].imshow(bilateral_conv(img, 7, 1, 1), cmap='gray')
ax[2].set_title('Biateral')
print('Bilateral convolution time: ', (cv2.getTickCount() - start_time) / cv2.getTickFrequency())
start_time = cv2.getTickCount()
ax[3].set_title('Non local means')
ax[3].imshow(non_local_means(img, 7, 7, 1, 1), cmap='gray')
print('Non local means convolution time: ', (cv2.getTickCount() - start_time) / cv2.getTickFrequency())
plt.show()